# Frequency vs complexity

In [ ]:
import utils
import altair as alt
import pandas as pd
from analysePhenotype import readAnalysed

Initial setup:

In [ ]:
df = readAnalysed('../cpp/out/out_12238_seeded2D_analysed.ftr')
nSampled = 1e6

In [ ]:
df = readAnalysed('../cpp/out/out_16989_analysed.ftr')
nSampled = 1000

In [ ]:
df = readAnalysed('/mnt/extraspace/joakim/210324/stochastic/2d/out_27175_analysed.ftr')
nSampled = 1e8
nDim = 2
method = 'stochastic'
alt.data_transformers.disable_max_rows()

In [ ]:
df

In [ ]:
chart = alt.Chart(df).mark_circle(size=60, opacity=0.6).encode(
    alt.X(alt.repeat("column"), type='quantitative', axis=alt.Axis(tickMinStep = 1)),
    alt.Y('freq:Q', scale=alt.Scale(type='log'), title="Frequency"),
    href='url:N',
    color=alt.Color('symmetries:Q', scale=alt.Scale(scheme="inferno")),
    tooltip=['url:N', 'count', 'freq:Q', 'minLz', 'minNc', 'minNt', 'rotsymms', 'reflsymms', 'invsymms']
).transform_calculate(
    freq='datum.count/{}'.format(nSampled),
    url='"https://akodiat.github.io/polycubes/?rule="+datum.minLz_r',
    symmetries='1+datum.rotsymms+datum.reflsymms+datum.invsymms'
).properties(
    width=200,
    height=200
).repeat(
    column=['minLz', 'minNc', 'minNt']
).properties(
title='Polycube frequency vs complexity, {:.1E} samples of {}D {}'.format(nSampled, nDim, method)
)
#chart.save('freq_vs_compl.html')
chart

In [ ]:
alt.Chart(df).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color=alt.Color('freq:Q', scale=alt.Scale(type='log', scheme="inferno")),
    href='url:N'
).properties(
    width=150,
    height=150
).transform_calculate(
    freq='datum.count/{}'.format(nSampled),
    url='"https://akodiat.github.io/polycubes/?rule="+datum.minLz_r',
    symmetries='1+datum.rotsymms+datum.reflsymms+datum.invsymms'
).repeat(
    row=['minLz', 'minNc', 'minNt', 'size', 'symmetries'],
    column=['minLz', 'minNc', 'minNt', 'size', 'symmetries']
).properties(
    title='Complexity comparison, {:.1E} samples of {}D {}'.format(nSampled, nDim, method)
)

In [ ]:
alt.Chart(df).mark_rect().encode(
    alt.X('minNc:N'),
    alt.Y('minNt:N'),
    color=alt.Color('count()', scale=alt.Scale(scheme="inferno"))
).properties(
    width=200,
    height=200
)


In [ ]:
alt.Chart(df).mark_rect().encode(
    alt.X('minNc:N'),
    alt.Y('minNt:N'),
    color=alt.Color('count()', scale=alt.Scale(scheme="inferno"))
).properties(
    width=200,
    height=200
)


In [ ]:
import h5py
f = h5py.File('/mnt/extraspace/joakim/210324/seeded/2d/out_10331.h5', 'r')

In [ ]:
f

Load data and plot

In [ ]:
# Load phenotype data
data = phenosToDataFrame({
    '2d': utils.loadPhenos('../cpp/out/2d_31c_5t_1e7/phenos'),
    '3d': utils.loadPhenos('../cpp/out/3d_31c_5t_1e7/phenos'),
    '1d': utils.loadPhenos('../cpp/out/1d_31c_5t_1e7/phenos'),
    
})
                          
# Serialize data on disk to avoid embedding everything
alt.data_transformers.enable('json');

In [ ]:
# Plot
chart = alt.Chart(data).mark_circle(size=60, opacity=0.7).encode(
    alt.X('compl', title="Number of colours", axis=alt.Axis(tickMinStep = 1)),
    alt.Y('freq', scale=alt.Scale(type='log'), title="Frequency"),
    alt.Color('dataset', scale=alt.Scale(scheme='set1')),
    href='url:N',
    size = 'size',
    tooltip=['rule','dataset', 'count', 'size']
).properties(
title='Phenotype frequency vs complexity'
).interactive()
chart.save('freq_vs_compl.html')
chart

In [ ]:
alt.vconcat(*[
    alt.Chart(phenosToDataFrame({
    name: utils.loadPhenos(path)
})).mark_circle(size=60, opacity=0.7).encode(
    alt.X('compl', title="Number of colours", axis=alt.Axis(tickMinStep = 1)),
    alt.Y('freq', scale=alt.Scale(type='log'), title="Frequency"),
    href='url:N',
    size='size',
    color='nCubeTypes',
    tooltip=['rule','count','nCubeTypes','size']
).properties(
title=name
) for name, path in [
    ('1d', '../cpp/out/1d_31c_5t_1e7/phenos'),
    ('2d', '../cpp/out/2d_31c_5t_1e7/phenos'),
    ('3d', '../cpp/out/3d_31c_5t_1e7/phenos')
]])

In [ ]:
(
    chart.transform_filter((alt.datum.dataset == '1d')) &
    chart.transform_filter((alt.datum.dataset == '2d')) &
    chart.transform_filter((alt.datum.dataset == '3d'))
)

In [ ]:
alt.Chart(data).transform_calculate(
    url='https://akodiat.github.io/polycubes?rule=' + alt.datum.rule
).mark_circle(size=60).encode(
    alt.X('nCubeTypes', title="Number of cube types", axis=alt.Axis(tickMinStep = 1)),
    alt.Y('freq', scale=alt.Scale(type='log'), title="Frequency"),
    color='dataset',
    href='url:N',
    #size = 'size',
    tooltip=['rule','dataset', 'count'] #,'size']
).properties(
title='Phenotype frequency vs complexity'
).interactive()

In [ ]:
alt.Chart(data).transform_calculate(
    url='https://akodiat.github.io/polycubes?rule=' + alt.datum.rule
).mark_circle(size=60).encode(
    alt.X('nCubeTypes', title="Number of cube types", axis=alt.Axis(tickMinStep = 1)),
    alt.Y('compl', title="Number of colours"),
    color='dataset',
    href='url:N',
    #size = 'size',
    tooltip=['rule','dataset', 'count'] #,'size']
).properties(
title='Phenotype frequency vs complexity'
).interactive()